# Project Description: Question Pair Classification

This project involves building a machine learning model to classify whether two given questions have the same meaning or not. The task is a **duplicate question detection** problem, where the goal is to determine if two questions asked in natural language are semantically equivalent or if they express different intents.

The dataset consists of two columns, **`question1`** and **`question2`**, which represent pairs of questions. The target variable **`is_duplicate`** indicates whether the two questions are duplicates (i.e., whether they have the same meaning or not). The goal is to train a model that can classify whether two questions are duplicates based on their textual content.

## Key Steps in the Project:

### 1. Data Preprocessing
The text data is cleaned and preprocessed, including:
- Lowercasing the text
- Removing unwanted characters (e.g., non-alphabetical characters)
- Tokenizing the text for BERT-based modeling

### 2. Model Selection
A **BERT-based model** is used for sequence classification. BERT (Bidirectional Encoder Representations from Transformers) is fine-tuned on this task to classify whether pairs of questions are duplicates or not.

### 3. Model Training
Using the Hugging Face `Trainer` API, the model is trained on the dataset with the following configurations:
- Number o2 training epoch32 5
- Batch size: 16
- Learning rate: 2e-5
- Evaluation strategy: Evaluation at the end of each epoch

### 4. Evaluation
The model’s performance is evaluated using metrics such as **accuracy**, **precision**, **recall**, and **F1-score**, with a particular focus on correctly identifying duplicate questions.

### 5. Output
After training, the model predicts whether pairs of questions are duplicates or not. The results are then evaluated and analyzed.

## Objective:
- To create an effective question pair classification model that can identify semantically similar or duplicate questions. This can be used in various natural language processing (NLP) applications such as question answering systems, FAQ generation, and duplicate detection in forums or search engines.

## Tools and Libraries Used:
- **Pandas** for data manipulation and preprocessing
- **Transformers** from Hugging Face for BERT tokenization and model handling
- **PyTorch** for building and training the neural network
- **scikit-learn** for evaluation metrics and data splitting


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers torch scikit-learn

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [3]:
data = pd.read_csv('/content/quora_question_pairs_train_.csv')

data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,332278,459256,459257,The Iliad and the Odyssey in the Greek culture?,How do I prove that the pairs of three indepen...,0
1,196656,297402,297403,What is practical management and what is strat...,What are the practical aspects of strategic ma...,0
2,113125,184949,184950,How useful is MakeUseOf Answers?,Is there any Q&A site that is not Yahoo answer...,0
3,266232,101283,163744,Which is the best place to reside in India and...,Which ia the best place to visit in India?,0
4,122738,17811,27517,Why do so many people ask questions on Quora t...,Why don't many people posting questions on Quo...,1


In [4]:
data['question1'] = data['question1'].fillna('').astype(str)
data['question2'] = data['question2'].fillna('').astype(str)

In [5]:
data.shape

(323432, 6)

# EDA

is_duplicate
0    204022
1    119410
Name: count, dtype: int64


In [ ]:
print(data['is_duplicate'].value_counts())

# Visualisation of the distribution of classes (histogram)
data['is_duplicate'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Distribution of Duplicate Questions')
plt.xlabel('is_duplicate')
plt.ylabel('Count')
plt.show()

In [7]:
text = ' '.join(data['question1'].fillna('') + ' ' + data['question2'].fillna(''))

# Create a word cloud
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    colormap='Blues'  
).generate(text)

# Word cloud visualisation
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  
plt.title('Word Cloud of Question Pairs')
plt.show()

# Pre-processing text

In [6]:
# Function for pre-processing text
def preprocess_text(text):
    text = text.lower()
    # Remove special characters (punctuation, numbers, extra spaces)
    text = re.sub(r'[^a-z\s]', '', text)  
    text = ' '.join(text.split())

    return text

In [8]:
data['question1'] = data['question1'].apply(preprocess_text)
data['question2'] = data['question2'].apply(preprocess_text)

print(data[['question1', 'question2']].head())

                                           question1  \
0     the iliad and the odyssey in the greek culture   
1  what is practical management and what is strat...   
2                    how useful is makeuseof answers   
3  which is the best place to reside in india and...   
4  why do so many people ask questions on quora t...   

                                           question2  
0  how do i prove that the pairs of three indepen...  
1  what are the practical aspects of strategic ma...  
2  is there any qa site that is not yahoo answers...  
3          which ia the best place to visit in india  
4  why dont many people posting questions on quor...  


In [9]:
data = data.drop(columns=["id", "qid1", "qid2"])

In [10]:
data.head()

,question1,question2,is_duplicate
0,the iliad and the odyssey in the greek culture,how do i prove that the pairs of three indepen...,0
1,what is practical management and what is strat...,what are the practical aspects of strategic ma...,0
2,how useful is makeuseof answers,is there any qa site that is not yahoo answers...,0
3,which is the best place to reside in india and...,which ia the best place to visit in india,0
4,why do so many people ask questions on quora t...,why dont many people posting questions on quor...,1


# Using the Bert tokeniser

In [ ]:
# Separation into training and test sets
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

# Load the BERT tokeniser
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['question1'], examples['question2'], truncation=True, padding=True, max_length=128)

train_encodings = tokenizer(list(train_df['question1']), list(train_df['question2']), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_df['question1']), list(test_df['question2']), truncation=True, padding=True, max_length=128)

# Creating datasets for Trainer
class QADataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = QADataset(train_encodings, list(train_df['is_duplicate']))
test_dataset = QADataset(test_encodings, list(test_df['is_duplicate']))

# Using the BERT model for classification

In [15]:
# Load a pre-trained BERT model for classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
# Setting up training parameters
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/bert_results',          # Where to save the results
    num_train_epochs=3,              # Number of epochs
    per_device_train_batch_size=8,   # The size of the batch for training
    per_device_eval_batch_size=8,    # The size of the batch for evaluation
    warmup_steps=500,                # Number of steps for warmup
    weight_decay=0.01,               # Regularisation
    logging_dir='/content/drive/MyDrive/bert_logs',            # Logging
    logging_steps=10,
    evaluation_strategy="epoch",     # Evaluation of the model after each epoch
)

# Creating a Trainer
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,           
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
import os

# Specify the path to the folder on Google Drive to store W&B data
os.environ["WANDB_DIR"] = "/content/drive/MyDrive/wandb"

import wandb
wandb.init(project='Quora_classification')

# MODEL TRAINING

In [17]:
# Model training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.610000,0.660057
2,0.715700,0.657676
3,0.679900,0.657870


TrainOutput(global_step=97032, training_loss=0.6272115598375362, metrics={'train_runtime': 6653.5679, 'train_samples_per_second': 116.664, 'train_steps_per_second': 14.583, 'total_flos': 5.10590025143424e+16, 'train_loss': 0.6272115598375362, 'epoch': 3.0})

# Model evaluation

In [18]:
# Evaluation of the model on the test set
predictions = trainer.predict(test_dataset)

# Convert predictions to class labels (0 or 1)
preds = predictions.predictions.argmax(axis=-1)

# Assessment of accuracy and output of the classification report
print(f"Accuracy: {accuracy_score(test_df['is_duplicate'], preds)}")
print(classification_report(test_df['is_duplicate'], preds))

Accuracy: 0.632414550064155
              precision    recall  f1-score   support

           0       0.63      1.00      0.77     40909
           1       0.00      0.00      0.00     23778

    accuracy                           0.63     64687
   macro avg       0.32      0.50      0.39     64687
weighted avg       0.40      0.63      0.49     64687



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## The model gives inaccurate results. Let's try to improve the model by adding learning_rate=2e-5 and increasing the batch size to 32

In [23]:

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/bert_results',
    num_train_epochs=2,               
    per_device_train_batch_size=32,   
    per_device_eval_batch_size=32,    
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/bert_logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    learning_rate=2e-5,               
    save_strategy="epoch",               
    load_best_model_at_end=True,     
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# Model training
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.256700,0.267974
2,0.184200,0.265034


TrainOutput(global_step=16172, training_loss=0.266076344319449, metrics={'train_runtime': 2750.8173, 'train_samples_per_second': 188.122, 'train_steps_per_second': 5.879, 'total_flos': 3.40393350095616e+16, 'train_loss': 0.266076344319449, 'epoch': 2.0})

In [25]:
# Evaluation of the model on the test set
predictions = trainer.predict(test_dataset)

# Convert predictions to class labels (0 or 1)
preds = predictions.predictions.argmax(axis=-1)

# Assessment of accuracy and output of the classification report
print(f"Accuracy: {accuracy_score(test_df['is_duplicate'], preds)}")
print(classification_report(test_df['is_duplicate'], preds))

Accuracy: 0.8933943450770634
              precision    recall  f1-score   support

           0       0.92      0.91      0.91     40909
           1       0.84      0.87      0.86     23778

    accuracy                           0.89     64687
   macro avg       0.88      0.89      0.89     64687
weighted avg       0.89      0.89      0.89     64687



## Model Performance Improvement
After fine-tuning the model with optimized hyperparameters, including a reduced learning rate, increased batch size, and more training epochs, the model performance has significantly improved.

## Summary of Improvements:
- Accuracy increased from 63% to 89%.
- Precision for class 1 improved from 0% to 0.84, and recall for class 1 improved from 0% to 0.87.
- The F1-score for class 1 improved from 0 to 0.86.
  
Overall, the model now demonstrates better precision, recall, and F1-score, ensuring more reliable predictions for both classes.
This improvement validates the effectiveness of hyperparameter tuning and indicates that the model is now much better equipped to handle the task of identifying duplicate question pairs.